In [ ]:
import pandas as pd
import re
from tqdm import tqdm
from langchain_ollama.llms import OllamaLLM
import random

In [ ]:
file_path = 'data.xlsx'
data = pd.read_excel(file_path)

In [ ]:

def extract_prices_from_text(text):
    if not isinstance(text, str):
        return None
    price_patterns = re.findall(r'\b\d{4,6}(?:-\d{4,6})?\b', text.replace(' ', ''))
    extracted_prices = []
    for price in price_patterns:
        if '-' in price:
            low, high = map(int, price.split('-'))
            extracted_prices.append(high)
        else:
            extracted_prices.append(int(price))
    return extracted_prices if extracted_prices else None

In [ ]:
def get_price_from_model(description, area):
    prompt = f"""
На основе следующего описания дома определите стоимость аренды за **одни сутки**:

{description}

Условия:
1. Если указана стоимость для **выходных дней** (пятница или суббота), используйте **максимальную стоимость для выходного дня**.
2. Если указана стоимость для **праздничных дней** или **новогодних праздников**, используйте **максимальную стоимость**, если это обосновано.
3. Если цены для будних дней (воскресенье-четверг) и выходных различаются, используйте стоимость для **выходного дня**.
4. Ответ должен быть **только числом** — стоимость аренды за сутки в рублях.
"""
    response = model(prompt)
    try:
        return int(re.search(r'\d+', response).group())
    except (ValueError, AttributeError):
        return None

def process_row(row):
    description = row.get('Описание', '')
    area = row.get('Площадь, м2', 0)
    initial_price = int(re.search(r'\d+', str(row.get('Цена', '0'))).group()) if pd.notna(row['Цена']) else 0
    extracted_prices = extract_prices_from_text(description)

    if extracted_prices:
        avg_price = sum(extracted_prices) // len(extracted_prices)
    else:
        avg_price = get_price_from_model(description, area) or initial_price

    if area < 100:
        avg_price = max(avg_price, 10000)
    elif 100 <= area < 300:
        avg_price = max(avg_price, 25000)
    else:
        avg_price = max(avg_price, 50000)

    if avg_price < initial_price:
        avg_price = initial_price + random.randint(2000, 5000)

    return round(avg_price / 1000) * 1000


In [ ]:
model = OllamaLLM(
    model="qwen2.5:72b-instruct-q4_0",
    temperature=0.1,
    device="cuda:0"
)

for index, row in tqdm(data.iterrows(), desc="Обработка данных", total=data.shape[0]):
    new_price = process_row(row)
    data.at[index, 'Цена'] = f"{new_price} руб/день"


In [ ]:
output_file = 'new_data.xlsx'
data.to_excel(output_file, index=False)

print("\nОбработанные данные:")
for index, row in data.iterrows():
    print(f"Описание: {row['Описание'][:100]}..., Новая цена: {row['Цена']}")

print(f"\nФайл успешно сохранён: {output_file}")
